In [1]:
__author__ = 'Monika Soraisam'
__email__ = 'monika.soraisam@noirlab.edu'

## Once DRAGONS is installed, it needs to be *set up*, which entails generating a config file. Presently, this config file only deals with calibrations. See https://dragons.readthedocs.io/projects/recipe-system-prog-manual/en/v3.1.0/appendices/caldb.html

In [2]:
import os
from pathlib import Path

## The default config file for dragons (with path) is `~/.dragons/dragonsrc`. For GOATS application it is best to set up a config file per data set being reduced. The location for the config file can be changed by setting an environment variable $DRAGONSRC. This has to be done before importing dragons libraries because the config file is determined during import.   

In [3]:
## Best to do the reduction in a subfolder of the location where the data for a given observation ID have been downloaded
data_path = "/home/monika.soraisam/Desktop/goats_dev/goats_data/data/ZTF23abuwrzn/GEM/GS-2021A-DD-102-9/" # an example 

## make a reduction folder 
reduction_path = Path(f"{data_path}/reduction")

if not reduction_path.exists():
    os.mkdir(reduction_path.as_posix())
    print (f"directory for dragons reduction created")

directory for dragons reduction created


In [4]:
mydb = "dragons_for_goats.db" 
mydb_path = reduction_path.as_posix() + '/' + mydb
print (mydb_path)


dragons_rc = reduction_path.as_posix() + '/dragonsrc'
print (dragons_rc)

with open(dragons_rc, "w") as f:
    f.write("[calibs]\ndatabases = {0} get store".format(mydb_path))

/home/monika.soraisam/Desktop/goats_dev/goats_data/data/ZTF23abuwrzn/GEM/GS-2021A-DD-102-9/reduction/dragons_for_goats.db
/home/monika.soraisam/Desktop/goats_dev/goats_data/data/ZTF23abuwrzn/GEM/GS-2021A-DD-102-9/reduction/dragonsrc


In [5]:
## we will put the config file in the reduction path

os.environ["DRAGONSRC"] = dragons_rc
print (f"$DRAGONSRC value is {os.environ.get('DRAGONSRC')}")

$DRAGONSRC value is /home/monika.soraisam/Desktop/goats_dev/goats_data/data/ZTF23abuwrzn/GEM/GS-2021A-DD-102-9/reduction/dragonsrc


## Next we create and initialize `mydb` from above as the calibration database (caldb)

In [6]:
# Let's import DRAGONS libraries

import astrodata
import gemini_instruments
from recipe_system import cal_service
from gempy.adlibrary import dataselect
from gempy.utils import logutils

In [7]:
caldb = cal_service.LocalDB(mydb_path, force_init=True) # this will create the db and initialize it with the tables for handling the calibrations

## Below is how you can access the calibration files registered in the caldb (calibration database)
for F in caldb.list_files():
    print (F)

## we can also remove existing calibrations in the caldb with the following 
## caldb.init(wipe=True)

2024-02-26 15:46:28 STDINFO  - Local database file /home/monika.soraisam/Desktop/goats_dev/goats_data/data/ZTF23abuwrzn/GEM/GS-2021A-DD-102-9/reduction/dragons_for_goats.db does not exist. Initializing.
Local database file /home/monika.soraisam/Desktop/goats_dev/goats_data/data/ZTF23abuwrzn/GEM/GS-2021A-DD-102-9/reduction/dragons_for_goats.db does not exist. Initializing.


In [10]:
## verify that the correct cal db is loaded
cal_service.load_config()
print (cal_service.get_db_path_from_config())

/home/monika.soraisam/Desktop/goats_dev/goats_data/data/ZTF23abuwrzn/GEM/GS-2021A-DD-102-9/reduction/dragons_for_goats.db


## During DRAGONS reduction, the "processed" calibration files will automatically be ingested in the caldb. But I'm manually adding some dummy procssed calibration for another observation ID by hand just to explore some of the methods of the `cal_service.LocalDB` class

In [16]:
fits_files = os.listdir(data_path)

In [17]:
fits_files

['S20210217S0120.fits',
 'S20210221S0282.fits',
 'S20210219S0075.fits',
 'S20210219S0078.fits',
 'S20210219S0076.fits',
 'S20210219S0077.fits',
 'S20210219S0079.fits',
 'S20210217S0118.fits',
 'S20210217S0117.fits',
 'S20210221S0278.fits',
 'S20210221S0280.fits',
 'S20210221S0286.fits',
 'S20210216S0123.fits',
 'S20210216S0122.fits',
 'S20210216S0121.fits',
 'S20210216S0120.fits',
 'S20210216S0119.fits',
 'S20210223S0104.fits',
 'S20210223S0105.fits',
 'S20210223S0106.fits',
 'S20210223S0107.fits',
 'S20210223S0108.fits',
 'S20210214S0042.fits',
 'S20210214S0041.fits',
 'S20210214S0040.fits',
 'S20210214S0039.fits',
 'S20210214S0038.fits',
 'S20210213S0060.fits',
 'S20210213S0059.fits',
 'S20210213S0058.fits',
 'S20210213S0057.fits',
 'S20210213S0056.fits',
 'S20210211S0072.fits',
 'S20210211S0071.fits',
 'S20210211S0070.fits',
 'S20210211S0069.fits',
 'S20210211S0068.fits',
 'S20210219S0102.fits',
 'S20210219S0101.fits',
 'S20210218S0029.fits',
 'reduction',
 'N20190331S0154_bias.fits

In [18]:
caldb.add_cal(f"{data_path}/N20190331S0154_bias.fits")

In [19]:
for F in caldb.list_files():
    print (F)

FileData(name='N20190331S0154_bias.fits', path='/home/monika.soraisam/Desktop/goats_dev/goats_data/data/ZTF23abuwrzn/GEM/GS-2021A-DD-102-9')


### Let's reinitialize the calibration database to explore how one can use an existing database

In [20]:
caldb = cal_service.LocalDB(mydb_path)

## Below is how you can access the calibration files registered in the caldb (calibration database)
for F in caldb.list_files():
    print (F)

FileData(name='N20190331S0154_bias.fits', path='/home/monika.soraisam/Desktop/goats_dev/goats_data/data/ZTF23abuwrzn/GEM/GS-2021A-DD-102-9')


**For whatever reason if the user wants to delete all the registered calibration files in the database, then one can do the following**

In [21]:
caldb.init(wipe=True)

for F in caldb.list_files():
    print (F)